In [5]:
import numpy as np

# Carga el archivo .npz
archive_title_npz = np.load('movie_titles.npz')
archive_movie_npz= np.load('movie_recommend.npz')

# Claves disponibles en el archivo de títulos
print("Claves disponibles en el archivo de títulos:", archive_title_npz.files)

# claves disponibles en el archivo de recomendaciones de películas
print("Claves disponibles en el archivo de recomendaciones de películas:", archive_movie_npz.files)




Claves disponibles en el archivo de títulos: ['titles']
Claves disponibles en el archivo de recomendaciones de películas: ['format', 'shape', 'data', 'row', 'col']
